In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import time
import os

In [20]:
# url
BASE_URL = "https://www.card-gorilla.com"
card_num = {
    "samsung": 1,
    "shinhan": 2,
    "kb": 3,
    "lotte": 4,
    "woori": 5,
    "hyundai": 7,
    "hana": 8,
    "nh": 9,
    "ibk": 10,
    "bc": 32
}
TEAM_DETAIL_URL = f"{BASE_URL}/team/detail/{card_num['shinhan']}"

In [13]:
# 1. 카드 목록 전체 로딩 (더보기 반복 클릭)

def load_all_cards():
    options = Options()
    options.add_argument("--headless=new")

    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=options)
    driver.get(TEAM_DETAIL_URL)
    time.sleep(2)

    wait = WebDriverWait(driver, 10)
    click_count = 0

    while True:
        try:
            # 더보기 버튼 감지
            more_button = wait.until(
                EC.presence_of_element_located((By.CLASS_NAME, "lst_more")))

            # 더 이상 버튼이 표시되지 않으면 중단
            if not more_button.is_displayed():
                print("[종료] 더보기 버튼이 더 이상 보이지 않음.")
                break

            # 버튼 클릭
            driver.execute_script("arguments[0].click();", more_button)
            click_count += 1
            print(f"[클릭] 더보기 버튼 클릭 {click_count}회")
            time.sleep(1.5)

        except Exception as e:
            print(f"[종료] 더보기 버튼 클릭 실패 또는 없음: {e}")
            break

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    return soup

In [16]:
# 2. 카드 리스트 추출

def get_card_list(soup):
    cards = soup.select("div.card_data")
    card_info_list = []

    for card in cards:
        try:
            # 발급 중단 카드 제외
            if card.select_one("p.txt_stop"):
                print("[-] 발급 중단 카드 - 스킵")
                continue

            name_tag = card.select_one("span.card_name")
            brand_tag = card.select_one("span.card_corp")
            href_tag = card.select_one("a.b_view")

            if not (name_tag and brand_tag and href_tag):
                continue

            name = name_tag.text.strip()
            brand = brand_tag.text.strip()
            detail_href = href_tag["href"]
            detail_url = BASE_URL + detail_href

            print(f"[+] {name} ({brand})")

            detail_info = get_card_detail(detail_url)

            card_info_list.append({
                "name": name,              # 카드명
                "brand": brand,            # 카드사
                "detail_url": detail_url,  # 카드 상세페이지 URL
                **detail_info
            })

            time.sleep(0.5)

        except Exception as e:
            print(f"[-] 카드 처리 중 오류 발생: {e}")
            continue

    return card_info_list

In [21]:
# 3. 카드 상세 정보 추출

def get_card_detail(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        options = Options()
        options.add_argument("--headless=new")  # 기존과 동일

        driver = webdriver.Chrome(service=Service(
            ChromeDriverManager().install()), options=options)
        driver.get(url)
        time.sleep(2)

        # [수정 1] 혜택 펼치기: 모든 dt 클릭
        dl_elements = driver.find_elements(By.CSS_SELECTOR, "div.bene_area dl")
        for dl in dl_elements:
            try:
                dt_tag = dl.find_element(By.TAG_NAME, "dt")
                driver.execute_script("arguments[0].click();", dt_tag)
                time.sleep(0.2)
            except Exception as e:
                print("[-] 혜택 클릭 실패:", e)

        # [수정 2] 클릭 후 HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")
        driver.quit()

        # 해외 결제 네트워크
        card_schemes_tag = soup.select("dd.c_brand span")
        card_schemes = [tag.text.strip()
                        for tag in card_schemes_tag] if card_schemes_tag else []

        # 연회비
        fee_domestic = ""
        fee_global = ""
        fees_tag = soup.select_one("dd.in_out")
        if fees_tag:
            spans = fees_tag.select("span")
            if len(spans) >= 2:
                domestic_text = spans[0].select_one("b")
                global_text = spans[1].select_one("b")
                fee_domestic = domestic_text.text.strip() if domestic_text else ""
                fee_global = global_text.text.strip() if global_text else ""

        # [기존 유지] 혜택 파싱
        benefits = []
        dl_list = soup.select("div.bene_area dl")
        for dl in dl_list:
            try:
                dt = dl.select_one("dt")
                category_tag = dt.select_one("p.txt1") if dt else None
                category = category_tag.text.strip() if category_tag else ""
                content = ""

                detail_box = dl.select_one("dd div.in_box")
                if not detail_box:
                    continue

                text_parts = []
                for child in detail_box.children:
                    text = child.get_text(strip=True)
                    if text:
                        text_parts.append(text)

                content = "\n".join(text_parts) if len(text_parts) != 0 else ""

                if category and content:
                    benefits.append({
                        "category": category,
                        "content": content,
                    })

            except Exception as e:
                print("[-] 혜택 파싱 실패:", e)

        return {
            "card_schemes": card_schemes,
            "fee_domestic": fee_domestic,
            "fee_global": fee_global,
            "benefits": benefits
        }

    except Exception as e:
        print(f"[-] 상세 페이지 파싱 오류: {e}")
        return {
            "card_schemes": [],
            "fee_domestic": "",
            "fee_global": "",
            "benefits": []
        }

In [ ]:
# 4. JSON 저장

def save_to_json(data, filename="신한.json"):
    save_path = os.path.join("data", "raw_hcw", filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"[+] JSON 저장 완료: {save_path}")

In [23]:
# 6. 실행

if __name__ == "__main__":
    soup = load_all_cards()
    card_data = get_card_list(soup)

    save_to_json(card_data)

cards = soup.select("div.card_data")
print(f"총 카드 개수: {len(cards)}")

[클릭] 더보기 버튼 클릭 1회
[클릭] 더보기 버튼 클릭 2회
[클릭] 더보기 버튼 클릭 3회
[클릭] 더보기 버튼 클릭 4회
[클릭] 더보기 버튼 클릭 5회
[클릭] 더보기 버튼 클릭 6회
[클릭] 더보기 버튼 클릭 7회
[종료] 더보기 버튼이 더 이상 보이지 않음.
[+] 신한카드 Mr.Life (신한카드)
[+] 신한카드 처음(ANNIVERSE) (신한카드)
[+] 신한카드 Discount Plan+ (신한카드)
[+] 신한카드 Deep Oil (신한카드)
[+] 신한카드 The BEST-F (신한카드)
[+] 신한카드 Air One (신한카드)
[+] 신한카드 RPM+ Platinum# (신한카드)
[+] K-패스 신한카드 (신한카드)
[+] 신한카드 Point Plan (신한카드)
[+] 신한카드 Deep On Platinum+ (신한카드)
[+] 메리어트 본보이™ 더 베스트 신한카드 (신한카드)
[+] 티머니 Pay & GO 신한카드 (신한카드)
[+] 신한카드 The BEST-X (신한카드)
[+] 신한카드 Air Platinum# (신한카드)
[+] 신한카드 The CLASSIC+ (신한카드)
[+] 신한카드 Point Plan+ (신한카드)
[+] 신한카드 Edu Plan+ (신한카드)
[+] 신한카드 구독 좋아요 (신한카드)
[+] 신한카드 Simple+ (신한카드)
[+] 네이버페이 라인프렌즈 신한카드 (신한카드)
[+] 토스페이 플러스 신한카드 (신한카드)
[+] 신한카드 후불하이패스+ (하이패스 전용) (신한카드)
[+] 신한카드 국민행복 (신한카드)
[+] #Pay 신한카드 (신한카드)
[+] 신한카드 The CLASSIC-Y (신한카드)
[+] 신한카드 Deep Taking (신한카드)
[+] 신한카드 Discount Plan (신한카드)
[+] 신한카드 EVerywhere (신한카드)
[+] 신한카드 Deep Store (신한카드)
[+] 신한카드 처음 (신한카드)
[+] 메리어트 본보이™ 더 클래식 신한카드 (신한카드)
[+] 